In [1]:
import pandas as pd
import seaborn as sb
import csv
from sympy import FiniteSet

import pysam
import os
import math
import random
import numpy as np

from pathlib import Path
from random import seed
from random import randint
from functools import reduce

from operator import add as concat

from itertools import combinations
from itertools import chain, combinations


In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

# Cargar las direcciones

In [3]:
ruta_reads = '/home/shaday/GIT/TDA-Metaganomas/ReadsSimulados/clavibacter/read_100M/'
ruta_genomas = "/home/shaday/GIT/TDA-Metaganomas/genomes/clavibacter"  ##Se queda igual
ruta_db = '/home/shaday/GIT/TDA-Metaganomas/output-tda/db-bowtie/clavibacter/'     ##Se queda igual
ruta_ids_db = "/home/shaday/GIT/TDA-Metaganomas/output-tda/db-bowtie/DB_clavibacter.txt"
ruta_output_bowtie = "/home/shaday/GIT/TDA-Metaganomas/output-tda/"
ruta_csv_files = ""

# Crear base de datos de bowtie ( correr solo una vez)

In [ ]:
%%bash -s "$ruta_genomas" "$ruta_db" "$ruta_ids_db"
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls $1/*fna | while read line; 
do 
    base=$(echo $line|cut -d'_' --complement -s -f1,2,3 |cut -d'.' -f1);
    #base=$(basename ${line} .fna)
    bowtie2-build $line $2$base;
    echo "$base" >> $3
done

In [5]:
contenido = list(os.listdir(ruta_genomas))
contenido = [na for na in contenido if "fna" in na]
contenido.sort()
contenido
contenido_names = [contenido[i][32:-4] for i in range(len(contenido))]
contenido_names


['capsici_1101',
 'insidiosus_ATCC_10253',
 'michi_contigs',
 'nebraskensis_419B',
 'sepedonicus_ATCC33113',
 'tessellarius_ATCC_33566']

In [6]:
for folder1 in contenido_names:
    os.makedirs(ruta_output_bowtie +folder1)
    os.makedirs(ruta_output_bowtie +folder1+'/sam')
    os.makedirs(ruta_output_bowtie +folder1+'/sorted')
  

## Correr bowtie

In [ ]:
%%bash -s "$ruta_reads" "$ruta_db" "$ruta_ids_db" "$ruta_output_bowtie"
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls $1*R1.fastq.gz | while read lectura
do
    id=$(basename ${lectura} _R1.fastq.gz)
    cat $3 | while read line
    do
        bowtie2 -x $2$line -1 $1${id}_R1.fastq.gz -2 $1${id}_R2.fastq.gz --no-unal -p 12 -S $4$id/sam/TP_${id}_to_${line}.sam  
    done
done;

Ordenar los archivos `sam`

In [8]:
for i in contenido_names:
    p=list(os.listdir(ruta_output_bowtie+i+"/sam/"))
    for j in p:
        pysam.sort("-o", ruta_output_bowtie+i+"/sorted/"+j, ruta_output_bowtie+i+"/sam/"+j)

# Construir CSV

In [11]:
for i in contenido_names:
    df=[]
    p=list(listdir_nohidden(ruta_output_bowtie+i+"/sorted/"))
    for j in p:
        samfile = pysam.AlignmentFile(ruta_output_bowtie+i+"/sorted/"+j, "rb")
        dftt=[]
        for read in samfile.fetch():
            r=str(read).split("\t")
            dftt.append(r[:6])
        samfile.close()
        dftt=pd.DataFrame(dftt, columns=['Qname','Flag', 'Rname', 'Pos','MapQ','CIGAR'])
        dftt['Organims']=j.split("_to_")[1].split(".")[0]
        df.append(dftt)
    df=pd.concat(df)
    df.to_csv(ruta_output_bowtie+'CSV/'+i+".csv")
        